In [13]:
# Import Dependencies
import pandas as pd
import plotly.express as px
# import hvplot.pandas
from sklearn.cluster import KMeans


In [4]:
# Display all columns
pd.set_option('display.max_columns', None)

In [155]:
# Load data into dataframe
df = pd.read_csv("csv_files/beer_info_from_db.csv")

In [156]:
df.head()

,beer_id,beer_name,beer_style,style_key,brewery,description,abv,ave_rating,min_ibu,max_ibu,astringency,body,alcohol,bitter,sweet,sour,salty,fruits,hoppy,spices,malty
0,1,Amber,Altbier,8,Alaskan Brewing Co.,"Notes:Richly malty and long on the palate, wit...",5.3,3.65,25,50,13,32,9,47,74,33,0,33,57,8,111
1,2,Double Bag,Altbier,8,Long Trail Brewing Co.,"Notes:This malty, full-bodied double alt is al...",7.2,3.90,25,50,12,57,18,33,55,16,0,24,35,12,84
2,3,Long Trail Ale,Altbier,8,Long Trail Brewing Co.,Notes:Long Trail Ale is a full-bodied amber al...,5.0,3.58,25,50,14,37,6,42,43,11,0,10,54,4,62
3,4,Doppelsticke,Altbier,8,Uerige Obergärige Hausbrauerei,Notes:,8.5,4.15,25,50,13,55,31,47,101,18,1,49,40,16,119
4,5,Scurry,Altbier,8,Off Color Brewing,Notes:Just cause it's dark and German doesn't ...,5.3,3.67,25,50,21,69,10,63,120,14,0,19,36,15,218


In [157]:
# Calculate average ibu
df["avg_ibu"] = (df.min_ibu + df.max_ibu) /2

In [158]:
beer_styles= []

In [159]:
# Function to get list of unique styles
def get_styles(x):
    if x not in beer_styles:
        beer_styles.append(x)

In [ ]:
beer_styles_count = []

In [160]:
# Function to add all styles to list
def get_style_count(x):
    beer_styles_count.append(x)

In [161]:
df.beer_style.apply(get_style_count)

0       None
1       None
2       None
3       None
4       None
        ... 
5551    None
5552    None
5553    None
5554    None
5555    None
Name: beer_style, Length: 5556, dtype: object

In [163]:
# Calculate number of items for each style
for item in beer_styles:
    counter = 0
    for x in beer_styles_count:
        if item == x:
            counter += 1
    print(item, counter)

Altbier 50
Barleywine - American 50
Bock - Eisbock 50
Barleywine - English 50
Bitter - English Extra Special / Strong Bitter (ESB) 50
Bière de Champagne / Bière Brut 50
Bitter - English 50
Blonde Ale - American 50
Blonde Ale - Belgian 50
Bock - Doppelbock 50
Bock - Maibock 50
Bock - Traditional 50
Bock - Weizenbock 50
Braggot 50
Chile Beer 50
Brett Beer 50
Farmhouse Ale - Sahti 50
Brown Ale - American 50
Happoshu 50
Brown Ale - Belgian Dark 50
Brown Ale - English 50
California Common / Steam Beer 50
Cream Ale 50
Lambic - Traditional 49
Dubbel 50
Farmhouse Ale - Bière de Garde 50
Gruit / Ancient Herbed Ale 50
Farmhouse Ale - Saison 50
Low Alcohol Beer 50
Fruit and Field Beer 50
Herb and Spice Beer 50
IPA - American 50
Porter - Smoked 50
IPA - Belgian 50
IPA - Black / Cascadian Dark Ale 50
IPA - Brut 50
IPA - English 50
Lager - European Dark 50
IPA - New England 50
IPA - Imperial 50
Kölsch 50
Kvass 49
Lager - Adjunct 50
Lager - American Amber / Red 50
Lager - American 50
Lager - European

In [162]:
df.beer_style.apply(get_styles)

0       None
1       None
2       None
3       None
4       None
        ... 
5551    None
5552    None
5553    None
5554    None
5555    None
Name: beer_style, Length: 5556, dtype: object

In [146]:
beer_styles_count= []

In [147]:
df.beer_style.apply(get_style_count)

0       None
1       None
2       None
3       None
4       None
        ... 
5551    None
5552    None
5553    None
5554    None
5555    None
Name: beer_style, Length: 5556, dtype: object

In [128]:
beer_styles

['Altbier',
 'Barleywine - American',
 'Barleywine - English',
 'Bitter - English Extra Special / Strong Bitter (ESB)',
 'Bière de Champagne / Bière Brut',
 'Bitter - English',
 'Bock - Maibock',
 'Bock - Weizenbock',
 'Braggot',
 'Chile Beer',
 'Brett Beer',
 'Farmhouse Ale - Sahti',
 'Brown Ale - American',
 'Happoshu',
 'Brown Ale - Belgian Dark',
 'Brown Ale - English',
 'California Common / Steam Beer',
 'Cream Ale',
 'Lambic - Traditional',
 'Dubbel',
 'Farmhouse Ale - Bière de Garde',
 'Gruit / Ancient Herbed Ale',
 'Farmhouse Ale - Saison',
 'Low Alcohol Beer',
 'Fruit and Field Beer',
 'Herb and Spice Beer',
 'IPA - New England',
 'Kölsch',
 'Kvass',
 'Lager - India Pale Lager (IPL)',
 'Lager - Vienna',
 'Lambic - Faro',
 'Lambic - Fruit',
 'Lambic - Gueuze',
 'Mild Ale - English Dark',
 'Mild Ale - English Pale',
 'Old Ale',
 'Rye Beer - Roggenbier',
 'Red Ale - Imperial',
 'Porter - Baltic',
 'Pumpkin Beer',
 'Red Ale - Irish',
 'Quadrupel (Quad)',
 'Red Ale - American Amber

In [181]:
beer_styles_condensed = ["IPA", "Porter", "Lager", "Pale Ale", "Pilsner", "Stout", "Wheat Beer", "Bock", 
                         "Blonde Ale", "Sour", "Lambic", "Brown Ale", "Barleywine", "Strong Ale", "Farmhouse Ale","Bitter", "Red Ale"]

In [196]:
def reduce_styles(style):
    for item in beer_styles_condensed:
        print(item, style)
        if item in style:
            return(item)
        else:
            return(style)

In [197]:
df["condensed_style"] = df.beer_style.apply(reduce_styles)

IPA Altbier
IPA Altbier
IPA Altbier
IPA Altbier
IPA Altbier
IPA Altbier
IPA Altbier
IPA Altbier
IPA Altbier
IPA Altbier
IPA Altbier
IPA Altbier
IPA Altbier
IPA Altbier
IPA Altbier
IPA Altbier
IPA Altbier
IPA Altbier
IPA Altbier
IPA Altbier
IPA Altbier
IPA Altbier
IPA Altbier
IPA Altbier
IPA Altbier
IPA Altbier
IPA Altbier
IPA Altbier
IPA Altbier
IPA Altbier
IPA Altbier
IPA Altbier
IPA Altbier
IPA Altbier
IPA Altbier
IPA Altbier
IPA Barleywine - American
IPA Barleywine - American
IPA Barleywine - American
IPA Altbier
IPA Altbier
IPA Altbier
IPA Altbier
IPA Altbier
IPA Altbier
IPA Altbier
IPA Altbier
IPA Altbier
IPA Altbier
IPA Altbier
IPA Altbier
IPA Altbier
IPA Altbier
IPA Barleywine - American
IPA Barleywine - American
IPA Barleywine - American
IPA Barleywine - American
IPA Barleywine - American
IPA Barleywine - American
IPA Barleywine - American
IPA Barleywine - American
IPA Barleywine - American
IPA Barleywine - American
IPA Barleywine - American
IPA Barleywine - American
IPA Barley

IPA Brett Beer
IPA Brett Beer
IPA Brett Beer
IPA Brett Beer
IPA Brett Beer
IPA Brett Beer
IPA Brett Beer
IPA Brett Beer
IPA Brett Beer
IPA Brown Ale - American
IPA Brown Ale - American
IPA Brown Ale - American
IPA Brown Ale - American
IPA Brown Ale - American
IPA Brown Ale - American
IPA Brown Ale - American
IPA Brown Ale - American
IPA Brown Ale - American
IPA Brown Ale - American
IPA Brown Ale - American
IPA Brown Ale - American
IPA Brown Ale - American
IPA Brown Ale - American
IPA Brown Ale - American
IPA Brown Ale - American
IPA Brown Ale - American
IPA Farmhouse Ale - Sahti
IPA Happoshu
IPA Brown Ale - American
IPA Brown Ale - American
IPA Brown Ale - American
IPA Brown Ale - American
IPA Brown Ale - American
IPA Brown Ale - American
IPA Brown Ale - American
IPA Brown Ale - American
IPA Brown Ale - American
IPA Brown Ale - American
IPA Brown Ale - American
IPA Brown Ale - American
IPA Brown Ale - American
IPA Brown Ale - American
IPA Brown Ale - American
IPA Happoshu
IPA Brown Ale

IPA Farmhouse Ale - Bière de Garde
IPA Farmhouse Ale - Bière de Garde
IPA Farmhouse Ale - Bière de Garde
IPA Farmhouse Ale - Bière de Garde
IPA Farmhouse Ale - Bière de Garde
IPA Farmhouse Ale - Bière de Garde
IPA Farmhouse Ale - Bière de Garde
IPA Farmhouse Ale - Bière de Garde
IPA Farmhouse Ale - Bière de Garde
IPA Farmhouse Ale - Bière de Garde
IPA Farmhouse Ale - Bière de Garde
IPA Farmhouse Ale - Bière de Garde
IPA Farmhouse Ale - Bière de Garde
IPA Farmhouse Ale - Bière de Garde
IPA Farmhouse Ale - Bière de Garde
IPA Farmhouse Ale - Bière de Garde
IPA Farmhouse Ale - Bière de Garde
IPA Farmhouse Ale - Bière de Garde
IPA Farmhouse Ale - Bière de Garde
IPA Farmhouse Ale - Bière de Garde
IPA Farmhouse Ale - Bière de Garde
IPA Farmhouse Ale - Bière de Garde
IPA Farmhouse Ale - Bière de Garde
IPA Farmhouse Ale - Bière de Garde
IPA Farmhouse Ale - Bière de Garde
IPA Farmhouse Ale - Bière de Garde
IPA Farmhouse Ale - Bière de Garde
IPA Farmhouse Ale - Bière de Garde
IPA Farmhouse Ale - 

IPA IPA - English
IPA IPA - English
IPA IPA - English
IPA IPA - English
IPA IPA - English
IPA IPA - English
IPA IPA - Imperial
IPA IPA - Imperial
IPA IPA - Imperial
IPA IPA - Imperial
IPA IPA - Imperial
IPA IPA - Imperial
IPA IPA - Imperial
IPA IPA - Imperial
IPA IPA - Imperial
IPA IPA - Imperial
IPA IPA - Imperial
IPA IPA - Imperial
IPA IPA - Imperial
IPA IPA - Imperial
IPA IPA - Imperial
IPA IPA - Imperial
IPA IPA - Imperial
IPA IPA - Imperial
IPA IPA - Imperial
IPA IPA - Imperial
IPA IPA - Imperial
IPA IPA - New England
IPA IPA - Imperial
IPA IPA - Imperial
IPA IPA - Imperial
IPA IPA - Imperial
IPA IPA - Imperial
IPA IPA - Imperial
IPA IPA - Imperial
IPA IPA - Imperial
IPA IPA - Imperial
IPA IPA - Imperial
IPA IPA - Imperial
IPA IPA - Imperial
IPA IPA - Imperial
IPA IPA - Imperial
IPA IPA - Imperial
IPA IPA - Imperial
IPA IPA - Imperial
IPA Kölsch
IPA IPA - Imperial
IPA IPA - Imperial
IPA IPA - Imperial
IPA IPA - Imperial
IPA IPA - Imperial
IPA IPA - Imperial
IPA IPA - Imperial
IPA 

IPA Lager - European Dark
IPA Lager - European Dark
IPA Lager - European Dark
IPA Lager - European Dark
IPA Lager - European Dark
IPA Lager - European Dark
IPA Lager - European Dark
IPA Lager - European Dark
IPA Lager - European Dark
IPA Lager - European Dark
IPA Lager - European Dark
IPA Lager - European Dark
IPA Lager - European Dark
IPA Lager - European Dark
IPA Lager - European Dark
IPA Lager - European Dark
IPA Lager - European Dark
IPA Lager - European Dark
IPA Lager - European Dark
IPA Lager - European Dark
IPA Lager - European Dark
IPA Lager - European Dark
IPA Lager - European Dark
IPA Lager - European Dark
IPA Lager - European Dark
IPA Lager - European Dark
IPA Lager - European Dark
IPA Lager - European Dark
IPA Lager - European Pale
IPA Lager - European Pale
IPA Lager - European Pale
IPA Lager - European Pale
IPA Lager - European Pale
IPA Lager - European Pale
IPA Lager - European Pale
IPA Lager - European Pale
IPA Lager - European Pale
IPA Lager - European Pale
IPA Lager - 

IPA Lager - Märzen / Oktoberfest
IPA Lager - Märzen / Oktoberfest
IPA Lager - Märzen / Oktoberfest
IPA Lager - Märzen / Oktoberfest
IPA Lager - Märzen / Oktoberfest
IPA Lager - Märzen / Oktoberfest
IPA Lager - Märzen / Oktoberfest
IPA Lager - Märzen / Oktoberfest
IPA Lager - Märzen / Oktoberfest
IPA Lager - Märzen / Oktoberfest
IPA Lager - Märzen / Oktoberfest
IPA Lager - Märzen / Oktoberfest
IPA Lager - Märzen / Oktoberfest
IPA Lager - Märzen / Oktoberfest
IPA Lager - Märzen / Oktoberfest
IPA Lager - Märzen / Oktoberfest
IPA Lager - Rauchbier
IPA Lager - Vienna
IPA Lager - Märzen / Oktoberfest
IPA Lager - Märzen / Oktoberfest
IPA Lager - Märzen / Oktoberfest
IPA Lager - Märzen / Oktoberfest
IPA Lager - Märzen / Oktoberfest
IPA Lager - Märzen / Oktoberfest
IPA Lager - Märzen / Oktoberfest
IPA Lager - Märzen / Oktoberfest
IPA Lager - Märzen / Oktoberfest
IPA Lager - Märzen / Oktoberfest
IPA Lager - Märzen / Oktoberfest
IPA Lager - Märzen / Oktoberfest
IPA Lager - Rauchbier
IPA Lager - R

IPA Old Ale
IPA Old Ale
IPA Old Ale
IPA Old Ale
IPA Old Ale
IPA Old Ale
IPA Old Ale
IPA Old Ale
IPA Old Ale
IPA Old Ale
IPA Old Ale
IPA Old Ale
IPA Old Ale
IPA Old Ale
IPA Old Ale
IPA Porter - English
IPA Old Ale
IPA Old Ale
IPA Old Ale
IPA Old Ale
IPA Old Ale
IPA Old Ale
IPA Old Ale
IPA Old Ale
IPA Old Ale
IPA Old Ale
IPA Old Ale
IPA Old Ale
IPA Old Ale
IPA Old Ale
IPA Old Ale
IPA Old Ale
IPA Old Ale
IPA Old Ale
IPA Old Ale
IPA Old Ale
IPA Old Ale
IPA Old Ale
IPA Old Ale
IPA Old Ale
IPA Old Ale
IPA Old Ale
IPA Pale Ale - American
IPA Pale Ale - American
IPA Pale Ale - American
IPA Pale Ale - American
IPA Pale Ale - American
IPA Pale Ale - American
IPA Pale Ale - American
IPA Pale Ale - American
IPA Pale Ale - American
IPA Pale Ale - American
IPA Pale Ale - American
IPA Pale Ale - American
IPA Pale Ale - American
IPA Pale Ale - American
IPA Pale Ale - American
IPA Pale Ale - American
IPA Pale Ale - American
IPA Pale Ale - American
IPA Pale Ale - American
IPA Pale Ale - American
IPA Pal

IPA Porter - Robust
IPA Porter - Robust
IPA Porter - Robust
IPA Porter - Robust
IPA Porter - Robust
IPA Porter - Robust
IPA Porter - Robust
IPA Porter - Robust
IPA Porter - Robust
IPA Porter - Robust
IPA Porter - Robust
IPA Porter - Robust
IPA Porter - Robust
IPA Porter - Smoked
IPA Porter - Smoked
IPA Porter - Smoked
IPA Porter - Smoked
IPA Porter - Smoked
IPA Porter - Smoked
IPA Porter - Smoked
IPA Porter - Smoked
IPA Porter - Smoked
IPA Porter - Smoked
IPA Porter - Smoked
IPA Porter - Smoked
IPA Porter - Smoked
IPA Porter - Smoked
IPA Porter - Smoked
IPA Porter - Smoked
IPA Porter - Smoked
IPA Porter - Smoked
IPA Porter - Smoked
IPA Porter - Smoked
IPA Porter - Smoked
IPA Porter - Smoked
IPA Porter - Smoked
IPA Porter - Smoked
IPA Porter - Smoked
IPA Porter - Smoked
IPA Porter - Smoked
IPA Porter - Smoked
IPA Porter - Smoked
IPA Porter - Smoked
IPA Porter - Smoked
IPA Porter - Smoked
IPA Porter - Smoked
IPA Porter - Smoked
IPA Porter - Smoked
IPA Porter - Smoked
IPA Porter - Smoked


IPA Sour - Flanders Oud Bruin
IPA Sour - Flanders Oud Bruin
IPA Sour - Flanders Oud Bruin
IPA Sour - Flanders Oud Bruin
IPA Sour - Flanders Oud Bruin
IPA Sour - Flanders Oud Bruin
IPA Sour - Flanders Oud Bruin
IPA Sour - Flanders Oud Bruin
IPA Sour - Flanders Oud Bruin
IPA Sour - Flanders Oud Bruin
IPA Sour - Flanders Oud Bruin
IPA Sour - Flanders Oud Bruin
IPA Sour - Flanders Oud Bruin
IPA Sour - Flanders Oud Bruin
IPA Sour - Flanders Oud Bruin
IPA Sour - Flanders Oud Bruin
IPA Sour - Flanders Oud Bruin
IPA Sour - Flanders Oud Bruin
IPA Sour - Flanders Oud Bruin
IPA Sour - Flanders Oud Bruin
IPA Sour - Flanders Oud Bruin
IPA Sour - Flanders Oud Bruin
IPA Sour - Flanders Oud Bruin
IPA Sour - Flanders Oud Bruin
IPA Sour - Flanders Oud Bruin
IPA Sour - Flanders Oud Bruin
IPA Sour - Flanders Oud Bruin
IPA Sour - Flanders Oud Bruin
IPA Sour - Flanders Red Ale
IPA Sour - Flanders Red Ale
IPA Sour - Flanders Red Ale
IPA Sour - Flanders Red Ale
IPA Sour - Flanders Red Ale
IPA Sour - Flanders 

IPA Strong Ale - American
IPA Strong Ale - American
IPA Strong Ale - American
IPA Strong Ale - American
IPA Strong Ale - American
IPA Strong Ale - American
IPA Strong Ale - American
IPA Strong Ale - American
IPA Strong Ale - American
IPA Strong Ale - American
IPA Strong Ale - American
IPA Strong Ale - American
IPA Strong Ale - American
IPA Strong Ale - American
IPA Strong Ale - American
IPA Strong Ale - American
IPA Strong Ale - American
IPA Strong Ale - American
IPA Strong Ale - American
IPA Strong Ale - American
IPA Strong Ale - American
IPA Strong Ale - American
IPA Strong Ale - American
IPA Strong Ale - American
IPA Strong Ale - American
IPA Strong Ale - American
IPA Strong Ale - American
IPA Strong Ale - American
IPA Strong Ale - American
IPA Strong Ale - Belgian Dark
IPA Strong Ale - American
IPA Strong Ale - American
IPA Strong Ale - American
IPA Strong Ale - American
IPA Strong Ale - American
IPA Strong Ale - American
IPA Strong Ale - American
IPA Strong Ale - Belgian Dark
IPA 

In [191]:
df.condensed_style.to_list()

['Altbier',
 'Altbier',
 'Altbier',
 'Altbier',
 'Altbier',
 'Altbier',
 'Altbier',
 'Altbier',
 'Altbier',
 'Altbier',
 'Altbier',
 'Altbier',
 'Altbier',
 'Altbier',
 'Altbier',
 'Altbier',
 'Altbier',
 'Altbier',
 'Altbier',
 'Altbier',
 'Altbier',
 'Altbier',
 'Altbier',
 'Altbier',
 'Altbier',
 'Altbier',
 'Altbier',
 'Altbier',
 'Altbier',
 'Altbier',
 'Altbier',
 'Altbier',
 'Altbier',
 'Altbier',
 'Altbier',
 'Altbier',
 'Barleywine - American',
 'Barleywine - American',
 'Barleywine - American',
 'Altbier',
 'Altbier',
 'Altbier',
 'Altbier',
 'Altbier',
 'Altbier',
 'Altbier',
 'Altbier',
 'Altbier',
 'Altbier',
 'Altbier',
 'Altbier',
 'Altbier',
 'Altbier',
 'Barleywine - American',
 'Barleywine - American',
 'Barleywine - American',
 'Barleywine - American',
 'Barleywine - American',
 'Barleywine - American',
 'Barleywine - American',
 'Barleywine - American',
 'Barleywine - American',
 'Barleywine - American',
 'Barleywine - American',
 'Barleywine - American',
 'Barleywi

In [182]:
for item in beer_styles_condensed: # Loops though condensed list
    for i in beer_styles: # Loops through original style list
        if item in i:
            beer_styles.remove(i)

In [183]:
beer_styles

['Altbier',
 'Bière de Champagne / Bière Brut',
 'Braggot',
 'Chile Beer',
 'Brett Beer',
 'Happoshu',
 'California Common / Steam Beer',
 'Cream Ale',
 'Dubbel',
 'Gruit / Ancient Herbed Ale',
 'Low Alcohol Beer',
 'Fruit and Field Beer',
 'Herb and Spice Beer',
 'Kölsch',
 'Kvass',
 'Mild Ale - English Dark',
 'Mild Ale - English Pale',
 'Old Ale',
 'Rye Beer - Roggenbier',
 'Pumpkin Beer',
 'Quadrupel (Quad)',
 'Rye Beer',
 'Scottish Ale',
 'Scotch Ale / Wee Heavy',
 'Smoked Beer',
 'Tripel',
 'Winter Warmer',
 'Wild Ale']

In [103]:
len(beer_styles)

106

In [6]:
df.columns

Index(['beer_id', 'beer_name', 'beer_style', 'style_key', 'brewery',
       'description', 'abv', 'ave_rating', 'min_ibu', 'max_ibu', 'astringency',
       'body', 'alcohol', 'bitter', 'sweet', 'sour', 'salty', 'fruits',
       'hoppy', 'spices', 'malty'],
      dtype='object')

In [82]:
new_df = df.drop(['beer_id', 'beer_name', 'beer_style', 'style_key', 'brewery',
       'description', 'ave_rating', "min_ibu", "max_ibu", "salty", "bitter", "alcohol"], axis=1)

In [83]:
new_df.head()

,abv,astringency,body,sweet,sour,fruits,hoppy,spices,malty,avg_ibu
0,5.3,13,32,74,33,33,57,8,111,37.5
1,7.2,12,57,55,16,24,35,12,84,37.5
2,5.0,14,37,43,11,10,54,4,62,37.5
3,8.5,13,55,101,18,49,40,16,119,37.5
4,5.3,21,69,120,14,19,36,15,218,37.5


In [84]:
# Initializing model with K = 3 
model = KMeans(n_clusters=5, random_state=5)
model

KMeans(n_clusters=5, random_state=5)

In [85]:
# Fitting model
model.fit(new_df)

KMeans(n_clusters=5, random_state=5)

In [86]:
# Get the predictions
predictions = model.predict(new_df)
print(predictions)

[2 0 2 ... 2 0 4]


In [87]:
# Add a new class column to the df_iris
new_df["class"] = model.labels_
new_df.head()

,abv,astringency,body,sweet,sour,fruits,hoppy,spices,malty,avg_ibu,class
0,5.3,13,32,74,33,33,57,8,111,37.5,2
1,7.2,12,57,55,16,24,35,12,84,37.5,0
2,5.0,14,37,43,11,10,54,4,62,37.5,2
3,8.5,13,55,101,18,49,40,16,119,37.5,0
4,5.3,21,69,120,14,19,36,15,218,37.5,0


In [88]:
style_df = df[["beer_style", "style_key"]]

In [89]:
combined_df = new_df.join(style_df)

In [90]:
combined_df.head(50)

,abv,astringency,body,sweet,sour,fruits,hoppy,spices,malty,avg_ibu,class,beer_style,style_key
0,5.30,13,32,74,33,33,57,8,111,37.5,2,Altbier,8
1,7.20,12,57,55,16,24,35,12,84,37.5,0,Altbier,8
2,5.00,14,37,43,11,10,54,4,62,37.5,2,Altbier,8
3,8.50,13,55,101,18,49,40,16,119,37.5,0,Altbier,8
4,5.30,21,69,120,14,19,36,15,218,37.5,0,Altbier,8
5,7.20,25,51,45,9,11,51,20,95,37.5,0,Altbier,8
6,6.00,22,45,62,25,34,60,4,103,37.5,2,Altbier,8
7,5.30,28,40,58,29,36,54,8,97,37.5,2,Altbier,8
8,5.00,18,49,73,22,21,37,4,98,37.5,0,Altbier,8
9,4.80,25,35,39,13,8,60,16,97,37.5,2,Altbier,8


In [91]:
# Plotting the clusters with three features
fig = px.scatter(combined_df, x="class", y="style_key", color="class", width=800)
# fig.update_layout(legend=dict(x=0,y=1))
fig.show()

In [94]:
# Plotting the clusters with three features
fig = px.scatter_3d(combined_df, x="abv", y="avg_ibu", z="body", color="class", width=800, hover="beer_style")
# fig.update_layout(legend=dict(x=0,y=1))
fig.show()

TypeError: scatter_3d() got an unexpected keyword argument 'hover'

In [64]:
inertia = []
k = list(range(1, 15))

In [65]:
# Looking for the best K
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(new_df)
    inertia.append(km.inertia_)

In [66]:
# Define a DataFrame to plot the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)

AttributeError: 'DataFrame' object has no attribute 'hvplot'